In [1]:
import networkx as nx
import spacy

from spacy.matcher import Matcher
from spacy.tokens import Span

nlp =spacy.load('en_core_web_md')

In [2]:
# reading the graph from python pickle
kgraph = nx.read_gpickle('onlineKhabar25March2022.gpickle')

In [3]:
#Entity Pair Extraction
def get_entities(entry):
  ent1 = ""
  ent2 = ""

  prev_tok_dep = ""     # dependency tag of previous token in the sentence
  prev_tok_text = ""    # previous token in the sentence

  
  prefix = ""
  modifier = ""


  for tok in nlp(entry):
    # if token is punctuation, move on to the next token
    if tok.dep_ != "punct":
      # check: token is a compound word
      if tok.dep_ == "compound":
        prefix = tok.text
        # if the previous word was also a 'compound' then adding the current word to it
        if prev_tok_dep == "compound":
          prefix = prev_tok_text + " "+ tok.text

      # check: token is a modifier
      if tok.dep_.endswith("mod") == True:
        modifier = tok.text
        # if the previous word was also a 'compound' then adding the current word to it
        if prev_tok_dep == "compound":
          modifier = prev_tok_text + " "+ tok.text

      # capture subject
      if tok.dep_.find("subj") == True:
        ent1 = modifier +" "+ prefix + " "+ tok.text
        # resetting the variables
        prefix = ""
        modifier = ""
        prev_tok_dep = ""
        prev_tok_text = ""

      # capture object
      if tok.dep_.find("obj") == True:
        ent2 = modifier +" "+ prefix + " "+ tok.text
        # resetting the variables
        prefix = ""
        modifier = ""
        prev_tok_dep = ""
        prev_tok_text = ""

      
      prev_tok_dep = tok.dep_
      prev_tok_text = tok.text

  return [ent1.strip(), ent2.strip()]

In [4]:
#Relation Extraction
def get_relation(entry):

  # creating a spaCy object
  doc = nlp(entry)

  # creating a matcher class object 
  matcher = Matcher(nlp.vocab)

  # defining the pattern 
  pattern = [{'DEP':'ROOT'},            # finds root word in the sentence
            {'DEP':'prep','OP':"?"},    # checks if it is followed by preposition
            {'DEP':'agent','OP':"?"},   # checks if it is followed by agent
            {'POS':'ADJ','OP':"?"}]     # checks adjective part of speech 

  # adding rule to the matcher
  matcher.add("match_1", [pattern])

  matches = matcher(doc)
  k = len(matches) - 1
  try:  
      span = doc[matches[k][1]:matches[k][2]] 
  #for empty strings
  except:
      span = ""
      return span
  return (span.text)

In [9]:
#answer question
def get_answer(question):
    words = []
    
    #tokenize , lemmatize and remove stop words from question
    tokens = nlp(question) 
    for token in tokens:
        if token.is_stop == True:
            continue
        words.append(token.lemma_)
    sentense = ' '.join(words)
    
    #get relation
    relation = get_relation(sentense)
    #get subject entity
    subject = get_entities(question)[0]
    #convert kgraph to dataframe
    df = nx.to_pandas_edgelist(kgraph)
    
    #iterate over df and compare to get results
    for index in df.index:
        if df.EDGE[index] == relation:
            if df.source[index] == subject:
                print(df.target[index])
            elif df.target[index] == subject:
                return(df.source[index])
        
    print("No answer could be found ")

In [10]:
#Question 1
get_answer("What says rape flower corn?")
#Answer: coat bio paint

rape flower corn say
coat bio paint
No answer could be found 


In [11]:
#Question 2 
get_answer("How many people did Chitwan storm kill?")
#Answer:3

Chitwan storm kill
3
No answer could be found 


In [8]:
#Question 3
get_answer("Who is the independent mayor candidate of Kathmandu?")
#Balen Shah

 Kathmandu
No answer could be found 
No answer could be found 
No answer could be found 
No answer could be found 
No answer could be found 
No answer could be found 
No answer could be found 
No answer could be found 
No answer could be found 
No answer could be found 
No answer could be found 
No answer could be found 
No answer could be found 
No answer could be found 
No answer could be found 
No answer could be found 
No answer could be found 
No answer could be found 
No answer could be found 
No answer could be found 
No answer could be found 
No answer could be found 
No answer could be found 
No answer could be found 
No answer could be found 
No answer could be found 
No answer could be found 
No answer could be found 
No answer could be found 
No answer could be found 
No answer could be found 
No answer could be found 
No answer could be found 
No answer could be found 
No answer could be found 
No answer could be found 
No answer could be found 
No answer could be found 
N